**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# Strict Policy

*strict* forbids more than the *restrictive*

https://kubernetes.io/docs/concepts/policy/pod-security-policy


In [1]:
kubectl apply -f - <<EOF
apiVersion: policy/v1beta1
kind: PodSecurityPolicy
metadata:
  name: strict
#  annotations:
#    seccomp.security.alpha.kubernetes.io/allowedProfileNames: 'docker/default,runtime/default'
#    apparmor.security.beta.kubernetes.io/allowedProfileNames: 'runtime/default'
#    seccomp.security.alpha.kubernetes.io/defaultProfileName:  'runtime/default'
#    apparmor.security.beta.kubernetes.io/defaultProfileName:  'runtime/default'
spec:
  privileged: false
  # Required to prevent escalations to root.
  allowPrivilegeEscalation: false
  # This is redundant with non-root + disallow privilege escalation,
  # but we can provide it for defense in depth.
  requiredDropCapabilities:
  - ALL
  # Allow core volume types.
  volumes:
    - 'configMap'
    - 'emptyDir'
    - 'projected'
    - 'secret'
    - 'downwardAPI'
    # Assume that persistentVolumes set up by the cluster admin are safe to use.
    - 'persistentVolumeClaim'
  hostNetwork: false
  hostIPC: false
  hostPID: false
  runAsUser:
    # Require the container to run without root privileges.
    rule: 'MustRunAsNonRoot'
  seLinux:
    # This policy assumes the nodes are using AppArmor rather than SELinux.
    rule: 'RunAsAny'
  supplementalGroups:
    rule: 'MustRunAs'
    ranges:
      # Forbid adding the root group.
      - min: 1
        max: 65535
  fsGroup:
    rule: 'MustRunAs'
    ranges:
      # Forbid adding the root group.
      - min: 1
        max: 65535
  readOnlyRootFilesystem: false
EOF

podsecuritypolicy.policy/strict created


In [ ]:
man capabilities

In [2]:
kubectl apply -f - <<EOF
kind: ClusterRole
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  name: psp-strict
rules:
- apiGroups:
  - extensions
  resources:
  - podsecuritypolicies
  resourceNames:
  - strict
  verbs:
  - use
EOF

clusterrole.rbac.authorization.k8s.io/psp-strict created


In [3]:
kubectl apply -f - <<EOF
apiVersion: rbac.authorization.k8s.io/v1beta1
kind: RoleBinding
metadata:
  name: strict
  namespace: default
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: psp-strict
subjects:
- kind: ServiceAccount
  name: default
  namespace: default
EOF

rolebinding.rbac.authorization.k8s.io/strict created


## Create an Nginx without privileges 

In [4]:
kubectl apply -f - <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-strict
  namespace: default
  labels:
    app: nginx
spec:
  replicas: 1
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginxinc/nginx-unprivileged
EOF

deployment.apps/nginx-strict created


In [6]:
kubectl get pods

NAME                                          READY   STATUS    RESTARTS   AGE
nginx-hostnetwork-deployment-db7654df-88wgs   1/1     Running   0          16m
nginx-other-deployment-6799fc88d8-27zdq       1/1     Running   0          5m56s
nginx-strict-747fbd9b9b-6bc5m                 1/1     Running   0          7s


In [ ]:
kubectl get all

## Which policy has been applied

how can we detect, which policy is in use for a given pod 

In [7]:
POD=$(kubectl get  pod | awk '/nginx-strict/{print $1}' | head -1)
echo $POD

nginx-strict-747fbd9b9b-6bc5m


In [8]:
kubectl get pods

NAME                                          READY   STATUS    RESTARTS   AGE
nginx-hostnetwork-deployment-db7654df-88wgs   1/1     Running   0          16m
nginx-other-deployment-6799fc88d8-27zdq       1/1     Running   0          6m4s
nginx-strict-747fbd9b9b-6bc5m                 1/1     Running   0          15s


In [9]:
kubectl get pod $POD -o jsonpath='{.metadata.annotations.kubernetes\.io/psp}'

permissive

## Correct the first, time wrong after deploying a permissive policy!

In [ ]:
kubectl delete deployment nginx-strict

In [ ]:
kubectl get rolebinding

In [ ]:
kubectl delete rolebinding restrictive permissive

In [ ]:
POD=$(kubectl get  pod | awk '/nginx-strict/{print $1}' | head -1)
echo $POD

In [ ]:
kubectl get pod $POD -o jsonpath='{.metadata.annotations.kubernetes\.io/psp}'

## Can we still create the *normal* deployment

In [10]:
kubectl apply -f - <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-deployment
  namespace: default
  labels:
    app: nginx
spec:
  replicas: 1
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginx
EOF

deployment.apps/nginx-deployment created


In [11]:
kubectl get deployments

NAME                           READY   UP-TO-DATE   AVAILABLE   AGE
nginx-deployment               0/1     1            0           2s
nginx-hostnetwork-deployment   1/1     1            1           22m
nginx-other-deployment         1/1     1            1           11m
nginx-strict                   1/1     1            1           6m8s


In [12]:
kubectl get pods

NAME                                          READY   STATUS    RESTARTS   AGE
nginx-deployment-6799fc88d8-cthtm             1/1     Running   0          6s
nginx-hostnetwork-deployment-db7654df-88wgs   1/1     Running   0          22m
nginx-other-deployment-6799fc88d8-27zdq       1/1     Running   0          12m
nginx-strict-747fbd9b9b-6bc5m                 1/1     Running   0          6m13s


In [14]:
kubectl describe pod nginx-deployment-6799fc88d8-cthtm 

Name:         nginx-deployment-6799fc88d8-cthtm
Namespace:    default
Priority:     0
Node:         minikube/192.168.49.2
Start Time:   Thu, 28 Jan 2021 13:13:59 +0000
Labels:       app=nginx
              pod-template-hash=6799fc88d8
Annotations:  kubernetes.io/psp: permissive
Status:       Running
IP:           172.17.0.5
IPs:
  IP:           172.17.0.5
Controlled By:  ReplicaSet/nginx-deployment-6799fc88d8
Containers:
  nginx:
    Container ID:   docker://5778d11df9e1fdec87ee1850f0bf5c3f780b417812ce99d576a1570131f173ab
    Image:          nginx
    Image ID:       docker-pullable://nginx@sha256:10b8cc432d56da8b61b070f4c7d2543a9ed17c2b23010b43af434fd40e2ca4aa
    Port:           <none>
    Host Port:      <none>
    State:          Running
      Started:      Thu, 28 Jan 2021 13:14:01 +0000
    Ready:          True
    Restart Count:  0
    Environment:    <none>
    Mounts:         <none>
Conditions:
  Type              Status
  Initialized       True 
  Ready             True 
  Co

In [15]:
kubectl get pods -o jsonpath='{.items[*].metadata.annotations.kubernetes\.io/psp}'

permissive permissive permissive permissive

In [ ]:
kubectl get rolebindings

In [ ]:
kubectl delete clusterrole psp-restrictive

## Try again

In [ ]:
kubectl delete deployment nginx-deployment

## Go back and redeploy the nginx deployment

In [ ]:
kubectl get pods

In [ ]:
POD=$(kubectl get  pod | awk '/nginx-deployment/{print $1}' | head -1)
echo $POD

In [ ]:
kubectl describe pod

In [ ]:
kubectl get all

## Rebind the role

Use the PodSecurityBasic and recreat the restrictive psp and rebind the role

In [ ]:
kubectl delete pod $POD

**deleting the pod is enough**

In [ ]:
kubectl get all

## Conclusion

* don't mess up the psp, clusterroles and bindings
* better one role per account